# Tool Agent

In [ ]:
!pip install python-dotenv langchain-community wikipedia

In [2]:
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

In [3]:
from typing import Optional, Dict, Any
from memory import Memory
from persona import Persona
from llm.generator import Generator
from llm.openai.service import OpenAIService
from states.final_answer.factory import FinalAnswerFactory
from states.state_node import StateNode, default_tools_handler
from tools.langchain.wikipedia_query_run import LangchainWikipediaQueryRun


llm = OpenAIService(api_key=api_key)
tools = [LangchainWikipediaQueryRun()]

final_answer_state = FinalAnswerFactory.get_state(Generator(llm))

def search_prompt_handler(persona: Persona, memory: Memory, tool_schemas: Optional[Dict[str, Any]]) -> str:
    return f'''{persona.prompt()}

Search for information using your tools to help solve the following problem for the user.

Problem:
"""
{memory.data.get("goal")}
"""

Here are the schemas for the tools you have access to, pick only one:
"""
{tool_schemas}
"""

Respond with the JSON input for the tool of your choice to best solve the problem.'''


search_state = StateNode(
    name="search",
    prompt_handler=search_prompt_handler,
    generation_handler=default_tools_handler(
        next_state=final_answer_state.name,
        exclude_from_scratch_pad=False
    ),
    generator=Generator(
        service=llm,
        use_json_model=True,
        temperature=0.1,
        max_tokens=512
    ),
    tools=tools,
)


In [4]:
from memory import Memory
import json
from agent import Agent, Query

from persona import Persona

persona = Persona(
    description="You're an autonomous agent who solves problems in natural language. Use your knowledge to best answer the question.")

agent = Agent.start(name="ToolAgent", persona=persona, memory=Memory(), state_nodes=[search_state, final_answer_state])

response = agent.ask(Query(initial_state="search",
                           goal="Search for information on autonomous planning and scheduling and give me a summary of it."))
print(json.dumps([step.model_dump() for step in response.steps], indent=2))

agent.stop()

[
  {
    "name": "search",
    "prompt": "You're an autonomous agent who solves problems in natural language. Use your knowledge to best answer the question.\n\nSearch for information using your tools to help solve the following problem for the user.\n\nProblem:\n\"\"\"\nSearch for information on autonomous planning and scheduling and give me a summary of it.\n\"\"\"\n\nHere are the schemas for the tools you have access to, pick only one:\n\"\"\"\n[{'tool_name': 'wikipedia_query_run', 'tool_description': 'A tool for querying the Wikipedia API.', 'tool_parameters': {'description': 'The input to search for factual information on Wikipedia.', 'properties': {'tool_name': {'description': 'The name of the tool you choose to use.', 'title': 'Tool Name', 'type': 'string'}, 'query': {'title': 'Query', 'type': 'string'}}, 'required': ['tool_name', 'query'], 'title': 'WikipediaQueryInput', 'type': 'object'}}]\n\"\"\"\n\nRespond with the JSON input for the tool of your choice to best solve the pr

In [ ]:
!pip install mermaid-py

In [6]:
import mermaid as md
from mermaid.graph import Graph

graph: Graph = Graph('example',"""
graph TD;
    goal --> search
    search --> final_answer
    final_answer --> exit

    classDef stateNode fill:#fff,stroke:#333,stroke-width:2px,color:#000;
""")
graphe: md.Mermaid = md.Mermaid(graph)
graphe